In [16]:
import pandas as pd
import strategy
import matplotlib.pyplot as plt

In [17]:
def backtest(df_skew):
    position = 0
    cash = 0
    last_price = None
    results = []

    for i, row in df_skew.iterrows():
        midprice = row['midprice']
        skew = row['error_zscore']
        if abs(skew - 1) >= 0.0005:  # trigger a trade if skew factor changed at least 5bp
            trade_size = 1  # for simplicity, always trade size of 1
            if skew > 1:  # buy
                cash -= midprice * trade_size
                position += trade_size
            else:  # sell
                cash += midprice * trade_size
                position -= trade_size
        result = {
            'time': i,
            'cash': cash,
            'position': position,
            'midprice': midprice,
            'skew': skew,
            'value': cash + position * midprice,
        }
        if last_price is not None:
            result['return'] = (result['value'] - last_price) / last_price
        results.append(result)
        last_price = result['value']

    return pd.DataFrame(results).set_index('time')


In [18]:
def compute_metrics(df):
    net_profit = df['cash'].iloc[-1] - df['cash'].iloc[0]
    gross_profit = df[df['cash'] > 0]['cash'].sum()
    gross_loss = df[df['cash'] < 0]['cash'].sum()
    profit_factor = gross_profit / abs(gross_loss)
    max_drawdown = df['cash'].cummax() - df['cash']
    sharpe_ratio = df['return'].mean() / df['return'].std()
    total_trades = len(df)
    profitable_trades = len(df[df['return'] > 0])
    percent_profitable = profitable_trades / total_trades * 100
    avg_trade_profit = net_profit / total_trades
    largest_winning_trade = df[df['return'] > 0]['return'].max()
    largest_losing_trade = df[df['return'] < 0]['return'].min()
    
    print(f"Net Profit: {net_profit}")
    print(f"Gross Profit: {gross_profit}")
    print(f"Gross Loss: {gross_loss}")
    print(f"Profit Factor: {profit_factor}")
    print(f"Max Drawdown: {max_drawdown.max()}")
    print(f"Sharpe Ratio: {sharpe_ratio}")
    print(f"Total Number of Trades: {total_trades}")
    print(f"Percentage of Profitable Trades: {percent_profitable}%")
    print(f"Average Trade Profit: {avg_trade_profit}")
    print(f"Largest Winning Trade: {largest_winning_trade}")
    print(f"Largest Losing Trade: {largest_losing_trade}")

In [19]:

df_public, skew = strategy.compute_strat_skew('SOL/USDT', 20)
df_backtest = backtest(df_public)

df_backtest


Current Kalman strategy skew factor: 0.9986872878595591


C:\Users\esteb\AppData\Local\Temp\ipykernel_11080\2343656355.py:27: RuntimeWarning: divide by zero encountered in scalar divide
  result['return'] = (result['value'] - last_price) / last_price


,cash,position,midprice,skew,value,return
time,,,,,,
2023-05-22 22:19:00,19.6905,-1,19.6905,0.458690,0.0000,NaN
2023-05-22 22:20:00,39.3750,-2,19.6845,-0.692233,0.0060,inf
2023-05-22 22:21:00,59.0395,-3,19.6645,-1.591199,0.0460,6.666667
2023-05-22 22:22:00,78.7065,-4,19.6670,-1.173287,0.0385,-0.163043
2023-05-22 22:23:00,98.3730,-5,19.6665,-0.136136,0.0405,0.051948
...,...,...,...,...,...,...
2023-05-22 23:54:00,1410.9610,-72,19.5330,0.249550,4.5850,-0.156704
2023-05-22 23:55:00,1430.4960,-73,19.5350,0.599771,4.4410,-0.031407
2023-05-22 23:56:00,1450.0220,-74,19.5260,-0.827247,5.0980,0.147940


Net Profit: 1469.3854999999999
Gross Profit: 70148.43950000001
Gross Loss: 0.0
Profit Factor: inf
Max Drawdown: 58.731500000000096
Sharpe Ratio: nan
Total Number of Trades: 92
Percentage of Profitable Trades: 53.2608695652174%
Average Trade Profit: 15.971581521739129
Largest Winning Trade: inf
Largest Losing Trade: -0.5079365079354194


C:\Users\esteb\AppData\Local\Temp\ipykernel_11080\2405316298.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  profit_factor = gross_profit / abs(gross_loss)


In [20]:
plt.figure(figsize=(14, 7))
plt.plot(df_backtest.index, df_backtest['cash'])
plt.title('Cash Evolution Over Time')
plt.xlabel('Time')
plt.ylabel('Cash')
plt.grid(True)
plt.show()

compute_metrics(df_backtest)